In [1]:
"""import xml.etree.ElementTree as ET
import os
import xmltodict
import torch
from torch.utils.data import Dataset
import transformers as T
import shutil
import random
from PIL import Image
import numpy as np"""

'import xml.etree.ElementTree as ET\nimport os\nimport xmltodict\nimport torch\nfrom torch.utils.data import Dataset\nimport transformers as T\nimport shutil\nimport random\nfrom PIL import Image\nimport numpy as np'

In [2]:
"""
def xml_handling(root='./archive', target_root='data'):
    root='./archive'
    target_root='data'
    root_label=root+'/annotations'
    root_img=root+'/images/'
    
    xml_files=os.listdir(root_label)    
    i=0
    for file in xml_files:
        if file[0]=='.':
            continue
        file=root_label+"/"+file
        train=False
        if random.random()<0.8:
            train=True
        tree = ET.parse(file)
        root = tree.getroot()
        base_name=os.path.basename(file)
        
        base_name,ext=os.path.splitext(base_name)
        result = ET.tostring(root,encoding='utf-8').decode('utf-8')
        dict_data=xmltodict.parse(result)
        data=dict_data['annotation']['object']
        if len(data)==2:
            continue
        src_img=root_img+base_name+'.png'
        img_path,label_path="",""
        if data['name']=='cat':
            note='Cat'+" "+str(data['bndbox']['xmin'])+" "+str(data['bndbox']['ymin'])+" "+str(data['bndbox']['xmax'])+" "+str(data['bndbox']['ymax'])
            if train:
                label_path=target_root+'/train/Cat/Label/'+base_name+'.txt'
                img_path=target_root+'/train/Cat/'+base_name+'.png'
            else:
                label_path=target_root+'/val/Cat/Label/'+base_name+'.txt'
                img_path=target_root+'/val/Cat/'+base_name+'.png'
        else:
            note='Dog'+" "+str(data['bndbox']['xmin'])+" "+str(data['bndbox']['ymin'])+" "+str(data['bndbox']['xmax'])+" "+str(data['bndbox']['ymax'])
            
            if train:
                label_path=target_root+'/train/Dog/Label/'+base_name+'.txt'
                img_path=target_root+'/train/Dog/'+base_name+'.png'
            else:
                
                label_path=target_root+'/val/Dog/Label/'+base_name+'.txt'
                img_path=target_root+'/val/Dog/'+base_name+'.png'
        shutil.copy2(src_img,img_path)
        print(label_path)
        with open(label_path,'w') as f:
            f.write(note)
        
        
"""

'\ndef xml_handling(root=\'./archive\', target_root=\'data\'):\n    root=\'./archive\'\n    target_root=\'data\'\n    root_label=root+\'/annotations\'\n    root_img=root+\'/images/\'\n    \n    xml_files=os.listdir(root_label)    \n    i=0\n    for file in xml_files:\n        if file[0]==\'.\':\n            continue\n        file=root_label+"/"+file\n        train=False\n        if random.random()<0.8:\n            train=True\n        tree = ET.parse(file)\n        root = tree.getroot()\n        base_name=os.path.basename(file)\n        \n        base_name,ext=os.path.splitext(base_name)\n        result = ET.tostring(root,encoding=\'utf-8\').decode(\'utf-8\')\n        dict_data=xmltodict.parse(result)\n        data=dict_data[\'annotation\'][\'object\']\n        if len(data)==2:\n            continue\n        src_img=root_img+base_name+\'.png\'\n        img_path,label_path="",""\n        if data[\'name\']==\'cat\':\n            note=\'Cat\'+" "+str(data[\'bndbox\'][\'xmin\'])+" "+str(da

In [1]:
"""class AnimalData:
    def __init__(self,root,transforms):
        self.root=root
        self.transforms=transforms
        classes=sort(list(os.listdir(root)))
        images=[]
        for c in classes:
            if c=='Label':
                continue
            for img in sort(list(os.listdir(c))):
                images.append(root+c+img)
        self.images=images
        label_dir=[c+'/Label' for c in classes]
        labels=[]
        for c in classes:
            for label in sort(list(os.listdir(label_dir))):
                labels.append(root+label_dir+label)
        self.labels=labels
    def __getitem__(self,idx):
        img =Image.open(self.img[idx]).convert("RGB")
        with open(self.labels,'r') as f:
            content= f.read()
            lines=contents.split(' ')
            for i in range(1, len(lines)):
                lines[i]=int(lines[i])
        
        """
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import random
import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_labels0=[]
test_labels0=[]
train_imgs0=[]
test_imgs0=[]
classes=['Bear', 'Bull', 'Cat', 'Cattle',
           'Cheetah', 'Chicken', 'Crocodile',
           'Deer', 'Dog', 'Fox', 'Goat', 'Hippopotamus', 
           'Horse', 'Jaguar', 'Leopard', 'Lion', 'Lynx',
           'Monkey', 'Mule', 'Ostrich', 'Owl', 
           'Pig', 'Raccoon', 'Raven', 'Rhinoceros', 
           'Sheep', 'Snake', 'Tiger', 'Turkey', 'Turtle', 'Zebra']
"""classes=['Cat','Dog']"""
encode={}
for i in range(len(classes)):
    encode[classes[i]]=i+1

def checkclass(dirname):
    
    for animal in classes:
        if dirname.endswith(animal):
            return True
    return False

for dirname, dirs, filenames in os.walk('./data/'):
    
    if 'train' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        
        for label in sorted(os.listdir(labels)):
            
            
            if label.endswith('txt'):
                train_labels0.append(labels+label)
        
        for image in sorted(filenames):
            
            
            if image.endswith('png') or image.endswith('jpg'):
                train_imgs0.append(dirname+'/'+image)
    if 'val' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        for label in sorted(os.listdir(labels)):
            if label.endswith('.txt'):
                test_labels0.append(labels+label)
        for image in sorted(filenames):
            if image.endswith('png')or image.endswith('jpg'):
                test_imgs0.append(dirname+'/'+image)


In [3]:
print(len(train_imgs0))
print(len(train_labels0))
print(len(test_imgs0))
print(len(test_labels0))
train_labels=[]
test_labels=[]
train_imgs=[]
test_imgs=[]
for i in range(len(train_labels0)):
    if i%14==0:
        
        train_labels.append(train_labels0[i])
        train_imgs.append(train_imgs0[i])
for i in range(len(test_labels0)):
    if i%20==0:
        test_labels.append(test_labels0[i])
        test_imgs.append(test_imgs0[i])
print(len(train_imgs))
print(len(train_labels))
print(len(test_imgs))
print(len(test_labels))


14620
14620
6864
6864
1045
1045
344
344


In [4]:
class AnimalDataset(object):
    def __init__(self, transforms,img_dir,label_dir):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = img_dir
        self.label=label_dir
#         self.labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))

    def __getitem__(self, idx):
        # load images ad masks
        file_image = self.imgs[idx]
        file_label = self.label[idx]
        img = Image.open(file_image).convert("RGB")
        target={}
        target['image_id']=torch.tensor([idx])
        labels=[]
        boxes=[]
        #Generate Label
        with open(file_label,'r')as f:
            lines=f.readlines()
            for line in lines:
                line_list=line.strip().split(' ')
                labels.append(encode[line_list[0]])
                boxes.append([float(i) for i in line_list[1:]])
                
            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)        
        
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        
        iscrowd = torch.zeros((len(classes),), dtype=torch.int64)

        target['boxes']=boxes
        target['labels']=labels
       
        target["area"] = area
        target["iscrowd"] = iscrowd
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [5]:

class AugmentImageBboxes(object):
    def __init__(self):
        pass
    
    def __call__(self, image, bboxes):
        # decide if flip or rotate
        if random.random() < 0.5:
            # Flip
            image = transforms.functional.hflip(image)
            h,w=image.shape[-2:]
            bboxes[:, [0, 2]] = h - bboxes[:, [2, 0]] 
        else:
            # Rotate by 90 degrees
            image = transforms.functional.rotate(image, 90)
            h, w = image.shape[-2:]
            bboxes[:, [0, 1, 2, 3]] = [w - bboxes[:, 3], bboxes[:, 0], h - bboxes[:, 1], w - bboxes[:, 0]]
        return image, bboxes

In [6]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

DEFAULT MODEL

In [7]:
def get_model_instance_segmentation(num_classes):
    
    # load a model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

    # replace the classifier with a new one, that has
    # num_classes which is user-defined
    num_classes = len(classes)+1  # 1 class (person) + background
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

    

In [8]:

from engine import train_one_epoch , evaluate
import utils


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = len(classes)+1
    # use our dataset and defined transformations
    train_data=AnimalDataset(get_transform(train=True),train_imgs,train_labels)
    test_data=AnimalDataset(get_transform(train=False),test_imgs,test_labels)

    # define training and validation data loaders
    train_loader = torch.utils.data.DataLoader(
        train_data, batch_size=4, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    test_loader = torch.utils.data.DataLoader(
        test_data, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)
    
    model.load_state_dict(torch.load('./model/fasterRCNN/fasterRCNN0.pt'))
    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.05,
                                momentum=0.9, weight_decay=0.005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        torch.save(model.state_dict(), "./model/fasterRCNN/fasterRCNN"+str(epoch+10)+".pt")
        # evaluate on the test dataset
        evaluate(model, train_loader, device=device)

    print("That's it!")

In [9]:
main()

1it [00:02,  2.23s/it]

Epoch: [0]  [  0/262]  eta: 0:09:44  lr: 0.000024  loss: 0.1786 (0.1786)  loss_classifier: 0.1066 (0.1066)  loss_box_reg: 0.0582 (0.0582)  loss_objectness: 0.0008 (0.0008)  loss_rpn_box_reg: 0.0130 (0.0130)  time: 2.2298  data: 0.3021  max mem: 3869


11it [00:10,  1.34it/s]

Epoch: [0]  [ 10/262]  eta: 0:03:52  lr: 0.000216  loss: 0.1373 (0.1561)  loss_classifier: 0.0638 (0.0762)  loss_box_reg: 0.0507 (0.0585)  loss_objectness: 0.0011 (0.0017)  loss_rpn_box_reg: 0.0141 (0.0197)  time: 0.9238  data: 0.0345  max mem: 6905


21it [00:18,  1.13it/s]

...

Epoch: [9]  [250/262]  eta: 0:00:09  lr: 0.000005  loss: 0.0685 (0.0785)  loss_classifier: 0.0276 (0.0329)  loss_box_reg: 0.0280 (0.0293)  loss_objectness: 0.0006 (0.0013)  loss_rpn_box_reg: 0.0096 (0.0150)  time: 0.7827  data: 0.0045  max mem: 8318


262it [03:29,  1.49it/s]

Epoch: [9]  [260/262]  eta: 0:00:01  lr: 0.000005  loss: 0.0783 (0.0785)  loss_classifier: 0.0249 (0.0328)  loss_box_reg: 0.0278 (0.0291)  loss_objectness: 0.0008 (0.0013)  loss_rpn_box_reg: 0.0184 (0.0154)  time: 0.8198  data: 0.0045  max mem: 8318
Epoch: [9]  [261/262]  eta: 0:00:00  lr: 0.000005  loss: 0.0771 (0.0784)  loss_classifier: 0.0231 (0.0327)  loss_box_reg: 0.0278 (0.0291)  loss_objectness: 0.0007 (0.0013)  loss_rpn_box_reg: 0.0146 (0.0154)  time: 0.7802  data: 0.0044  max mem: 8318


262it [03:29,  1.25it/s]


Epoch: [9] Total time: 0:03:29 (0.7979 s / it)
creating index...
index created!
Test:  [  0/262]  eta: 0:04:07  model_time: 0.5044 (0.5044)  evaluator_time: 0.0037 (0.0037)  time: 0.9464  data: 0.3954  max mem: 8318
Test:  [100/262]  eta: 0:00:59  model_time: 0.2988 (0.3427)  evaluator_time: 0.0024 (0.0028)  time: 0.3469  data: 0.0083  max mem: 8318
Test:  [200/262]  eta: 0:00:22  model_time: 0.3183 (0.3439)  evaluator_time: 0.0024 (0.0028)  time: 0.3568  data: 0.0083  max mem: 8318
Test:  [261/262]  eta: 0:00:00  model_time: 0.3247 (0.3413)  evaluator_time: 0.0028 (0.0028)  time: 0.3553  data: 0.0083  max mem: 8318
Test: Total time: 0:01:35 (0.3635 s / it)
Averaged stats: model_time: 0.3247 (0.3413)  evaluator_time: 0.0028 (0.0028)
Accumulating evaluation results...
DONE (t=0.21s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.311
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.480
 Average Precision  (AP) @[ I

In [ ]:
def get_model_instance_mobilenet(num_classes):
    
    backbone = torchvision.models.mobilenet_v2(pretrained=True).features
    # FasterRCNN needs to know the number of
    # output channels in a backbone. For mobilenet_v2, it's 1280
    # so we need to add it here
    backbone.out_channels = 1280

    # let's make the RPN generate 5 x 3 anchors per spatial
    # location, with 5 different sizes and 3 different aspect
    # ratios. We have a Tuple[Tuple[int]] because each feature
    # map could potentially have different sizes and
    # aspect ratios 
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                       aspect_ratios=((0.5, 1.0, 2.0),))

    # let's define what are the feature maps that we will
    # use to perform the region of interest cropping, as well as
    # the size of the crop after rescaling.
    # if your backbone returns a Tensor, featmap_names is expected to
    # be [0]. More generally, the backbone should return an
    # OrderedDict[Tensor], and in featmap_names you can choose which
    # feature maps to use.
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                    output_size=7,
                                                    sampling_ratio=2)

    # put the pieces together inside a FasterRCNN model
    model = FasterRCNN(backbone,
                       num_classes=2,
                       rpn_anchor_generator=anchor_generator,
                       box_roi_pool=roi_pooler)
    return model


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = len(classes)+1
# use our dataset and defined transformations
train_data=AnimalDataset(get_transform(train=True),train_imgs,train_labels)
test_data=AnimalDataset(get_transform(train=False),test_imgs,test_labels)

# define training and validation data loaders
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

# get the model using our helper function
model = get_model_instance_segmentationmobilenet(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    torch.save(model.state_dict(), "./model/fasterRCNN/fasterRCNN"+str(epoch+1)+".pt")
    # evaluate on the test dataset
    evaluate(model, train_loader, device=device)


In [16]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = len(classes)+1
# use our dataset and defined transformations
train_data=AnimalDataset(get_transform(train=True),train_imgs,train_labels)
test_data=AnimalDataset(get_transform(train=False),test_imgs,test_labels)

# define training and validation data loaders
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

# get the model using our helper function
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)
model.load_state_dict(torch.load('./model/fasterRCNN/fasterRCNN0.pt'))
evaluate(model,test_loader,device=device)

creating index...
index created!
Test:  [  0/344]  eta: 0:07:52  model_time: 1.2324 (1.2324)  evaluator_time: 0.0018 (0.0018)  time: 1.3731  data: 0.1328  max mem: 569
Test:  [100/344]  eta: 0:00:22  model_time: 0.0704 (0.0838)  evaluator_time: 0.0008 (0.0011)  time: 0.0790  data: 0.0024  max mem: 639
Test:  [200/344]  eta: 0:00:12  model_time: 0.0783 (0.0802)  evaluator_time: 0.0011 (0.0011)  time: 0.0839  data: 0.0026  max mem: 646
Test:  [300/344]  eta: 0:00:03  model_time: 0.0783 (0.0780)  evaluator_time: 0.0012 (0.0012)  time: 0.0808  data: 0.0025  max mem: 658
Test:  [343/344]  eta: 0:00:00  model_time: 0.0781 (0.0775)  evaluator_time: 0.0011 (0.0012)  time: 0.0808  data: 0.0024  max mem: 658
Test: Total time: 0:00:28 (0.0840 s / it)
Averaged stats: model_time: 0.0781 (0.0775)  evaluator_time: 0.0011 (0.0012)
Accumulating evaluation results...
DONE (t=0.13s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.200
 Average Precision  (AP) 

In [23]:
print(torch.cuda.memory_allocated(device="cuda")/ (1024**2), 'MB')
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated(device="cuda")/ (1024**2), 'MB')


10485.4501953125 MB
10485.4501953125 MB


In [39]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
dataset = AnimalDataset( get_transform(train=True),train_imgs,train_labels)
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=utils.collate_fn)
print('hi')
# For Training
images,targets = next(iter(data_loader))

images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)
# Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]

predictions = model(x)  
print(predictions)

hi
[{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>), 'labels': tensor([], dtype=torch.int64), 'scores': tensor([], grad_fn=<IndexBackward0>)}, {'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>), 'labels': tensor([], dtype=torch.int64), 'scores': tensor([], grad_fn=<IndexBackward0>)}]


In [11]:
num_epochs =1
model.to(device)
from tqdm import tqdm
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

len_dataloader = len(data_loader)

for epoch in range(num_epochs):
    model.train()
    i = 0    
    epoch_loss = 0
    for imgs, annotations in tqdm(data_loader):
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model([imgs[0]], [annotations[0]])
        losses = sum(loss for loss in loss_dict.values())        

        optimizer.zero_grad()
        
        losses.backward()
        optimizer.step() 
#         print(f'Iteration: {i}/{len_dataloader}, Loss: {losses}')
        epoch_loss += losses
    print(epoch_loss)

  1%|          | 9/914 [00:28<48:24,  3.21s/it]


IndexError: Target 3 is out of bounds.

In [ ]:
for imgs, annotations in data_loader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        break

In [ ]:
model.eval()
preds = model(imgs)
preds


In [ ]:
torch.save(model.state_dict(), "model.pth")

In [33]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/lqhung2001/.local/lib/python3.10/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +
